In [1]:
from qgis.core import *
from qgis.utils import *
from pyproj import Proj, transform
from qgis.gui import *
import sys
import utm
from PyQt4 import QtGui
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import time as t
%matplotlib

Using matplotlib backend: TkAgg


In [7]:

#QgsApplication.initQgis()
app = QgsApplication([],True, None)
app.setPrefixPath("/usr", True)
app.initQgis()

layer = QgsVectorLayer('/home/mandy8345/Map_3D/contourfiles/wells_test.shp', 'layer_name', 'ogr')
ite = layer.getFeatures()
i=0
#tof ='EPSG:4326'
inf = 'EPSG:3857'#layer.crs().authid()
tof ='EPSG:4326'
inProj = Proj(init=inf)
outProj = Proj(init=tof)
coords=[]
for feature in ite:
    # retrieve every feature with its geometry and attributes
    # fetch geometry
    geom = feature.geometry()
    x1,y1 = geom.asPoint().x(),geom.asPoint().y()
    poli = transform(inProj,outProj,x1,y1)
    coords.append(poli)
coords=np.array(coords)
                            

In [8]:
#print coords[1]
if not layer.isValid():
    print "Layer failed to load!"

In [9]:
#coords = df.as_matrix(columns=[aj,bh])


kms_per_radian = 6371.0088
epsilon = 1.5 / kms_per_radian

#epsilon parameter is the max distance (1.5 km in this example) 
#that points can be from each other to be considered a cluster

#min_samples parameter is the minimum cluster size 
#everything else gets classified as noise
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)]) 
#'clusters' is pandas series object
print 'Number of clusters:'+str(num_clusters);

Number of clusters:344


In [10]:
t_st=t.clock()
x=[];y=[]               
b=clusters
cm = plt.cm.get_cmap('jet')
xi=[];yi=[];dk=[];o=0;
for j in b:
    for i in j:
       xi.append(i[0])
       yi.append(i[1])
       dk.append(len(j))
    dr=len(j)
    if dr>o:
        o=dr
lon1=max(xi)
lon2=min(xi)
lat=0
zone1=utm.from_latlon(lat,lon1)
zone2=utm.from_latlon(lat,lon2) 
fig, ax = plt.subplots()#(figsize=(90,50))
fe=ax.scatter(xi,yi,c=dk,vmin=1,vmax=o,s=20,cmap=cm,label="go")
tfd=fe.get_array()
tr=plt.colorbar(fe)
#tr.ax.tick_params(direction='in', pad=0)
tr.ax.set_ylabel('number of points in the cluster')
#plotfile = "test_"+str(drty)
#plt.savefig(plotfile);drty+=1
print t.clock()-t_st
plt.show() 

0.544712
